## Requesting and combining multiple Jupyter notebooksOpvragen en combineren van meerdere jupyter notebooks



### Retrieve JSON objects for the desired locations
Based on the location codes, we need more information about the locations.
The API Request body will need to be filled with information about the locations to inform the API about the data we need from it.

In [1]:
import pandas as pd
import json

file = './../Data/Raw/rws_catalog.json'
with open(file, 'r') as f:
    data = json.load(f)

locatie_codes = ['J6', 'K14', 'SPY', 'BG2']
df_LocatieLijst = pd.DataFrame(data['LocatieLijst'])
df_LocatieLijstFiltered = df_LocatieLijst[df_LocatieLijst['Code'].isin(locatie_codes)].drop_duplicates(subset='Code', keep='first')

print(df_LocatieLijstFiltered)


      Locatie_MessageID Coordinatenstelsel              X             Y  \
2042               5377              25831  496708.794769  5.963122e+06   
2047              12760              25831  603084.756918  5.813701e+06   
2053              14674              25831  542240.426001  5.902123e+06   
2386              15429              25831  542904.767181  5.735254e+06   

                       Naam Code  
2042            J6 platform   J6  
2047  IJgeul stroommeetpaal  SPY  
2053           K14 platform  K14  
2386  Brouwershavense Gat 2  BG2  


### Retrieve the parameters for measurements
To retrieve a bulk of data, it is useful to obtain all other information based on the code of a parameter. This information is needed to fill the JSON request body, so the API knows which data needs to be retrieved.

In [2]:
import pandas as pd
import json


parameterGrootheid_codes = ['WINDRTG', 'WINDSHD', 'Tm02', 'Hm0','Th0']

AquoMetadataLijst = pd.json_normalize(data['AquoMetadataLijst'])
df_AquoMetadataLijst = pd.DataFrame(AquoMetadataLijst)
df_AquoMetadataLijstFiltered = df_AquoMetadataLijst[df_AquoMetadataLijst['Grootheid.Code'].isin(parameterGrootheid_codes)]

df_AquoMetadataLijstFiltered.head()

,AquoMetadata_MessageID,Parameter_Wat_Omschrijving,Compartiment.Code,Compartiment.Omschrijving,Grootheid.Code,Grootheid.Omschrijving
17,18,Windrichting Lucht t.o.v. ware Noorden in graad,LT,Lucht,WINDRTG,Windrichting
19,20,Windsnelheid Lucht t.o.v. Mean Sea Level in m/s,LT,Lucht,WINDSHD,Windsnelheid
61,62,Significante golfhoogte in het spectrale domei...,OW,Oppervlaktewater,Hm0,Significante golfhoogte in het spectrale domein
92,93,Gemiddelde golfrichting in het spectrale domei...,OW,Oppervlaktewater,Th0,Gemiddelde golfrichting in het spectrale domein
97,98,Golfperiode bepaald uit de spectrale momenten ...,OW,Oppervlaktewater,Tm02,Golfperiode bepaald uit de spectrale momenten ...


### Loop through all the requests
In this code, you select a start and end date for which you want to retrieve your data. Then, you loop through the list of locations and parameters to retrieve all measurements for this period.

In [4]:

import requests

begin_datum ="2025-01-01T10:00:00.000+01:00"
eind_datum ="2025-01-27T20:00:00.000+01:00"

collect_observations = ('https://waterwebservices.rijkswaterstaat.nl/' +
                        'ONLINEWAARNEMINGENSERVICES_DBO/' +
                        'OphalenWaarnemingen')

for i in range(0,len(locatie_codes)):
  for j in range(0,len(parameterGrootheid_codes)):

    request = {"Locatie":{"X": float(df_LocatieLijstFiltered['X'].iloc[i]),
                          "Y": float(df_LocatieLijstFiltered['Y'].iloc[i]),
                          "Naam": df_LocatieLijstFiltered['Naam'].iloc[i],
                          "Code": df_LocatieLijstFiltered['Code'].iloc[i]},
                "AquoPlusWaarnemingMetadata":{
                  "AquoMetadata":{"Compartiment":{"Code":df_AquoMetadataLijstFiltered['Compartiment.Code'].iloc[j]}, 
                                  "Grootheid":{"Code":df_AquoMetadataLijstFiltered['Grootheid.Code'].iloc[j]}}},
                "Periode":{
                  "Begindatumtijd":begin_datum,
                  "Einddatumtijd":eind_datum}}
    
    print("RWS_{0}_{1}".format(df_LocatieLijstFiltered['Code'].iloc[i],
                               df_AquoMetadataLijstFiltered['Grootheid.Code'].iloc[j]))

    resp = requests.post(collect_observations, json=request) # Zet deze aan om gegevens op te halen
    result = resp.json()
    output_file = r"./../Data/Raw/RWS_{0}_{1}.json".format(df_LocatieLijstFiltered['Code'].iloc[i], df_AquoMetadataLijstFiltered['Grootheid.Code'].iloc[j])
    with open(output_file, 'w') as f:
        json.dump(result, f, indent=4)

RWS_J6_WINDRTG
RWS_J6_WINDSHD
RWS_J6_Hm0
RWS_J6_Th0
RWS_J6_Tm02
RWS_SPY_WINDRTG
RWS_SPY_WINDSHD
RWS_SPY_Hm0
RWS_SPY_Th0
RWS_SPY_Tm02
RWS_K14_WINDRTG
RWS_K14_WINDSHD
RWS_K14_Hm0
RWS_K14_Th0
RWS_K14_Tm02
RWS_BG2_WINDRTG
RWS_BG2_WINDSHD
RWS_BG2_Hm0
RWS_BG2_Th0
RWS_BG2_Tm02
